In [1]:
import os 
from malaria_cnn import logger

In [2]:
os.chdir('../')

In [3]:
%pwd

'/home/fibonacci/projects/Detecting_Malaria_CNN'

In [4]:
import tensorflow as tf
model=tf.keras.models.load_model('artifacts/training/model.h5')

2024-05-19 09:55:11.821333: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-05-19 09:55:17,596: NumExpr defaulting to 4 threads.: INFO
2024-05-19 09:55:19,731: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.: WARNING


In [19]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [9]:
from malaria_cnn.constant import *
from malaria_cnn.utils.common import read_yaml,create_directories,save_json

In [15]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model='artifacts/training/model.h5',
            training_data='artifacts/data_ingestion/cell_images/cell_images',
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE[:-1],
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config


In [12]:
from urllib.parse import urlparse

In [16]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    
    def _valid_generator(self):
        datagenerator_kwargs=dict(
            rescale=1./255,
            validation_split=0.30
        )
        dataflow_kwargs=dict(
            target_size=self.config.params_image_size,
            batch_size=self.config.params_batch_size,
            interpolation='bilinear'
        )
        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )
    @staticmethod
    def load_model(path:Path)->tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        model=self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score=model.evaluate(self.valid_generator)

    def save_score(self):
        scores={"loss":self.score[0],'accuracy':self.score[1]}
        save_json(path=Path('scores.json'),data=scores)


In [20]:
try:
    config=ConfigurationManager()
    eval=Evaluation(config.get_validation_config())
    eval.evaluation()
    eval.save_score()
except Exception as e:
    print(e)

2024-05-19 10:07:30,736: yaml file:config/config.yaml load successfully: INFO
2024-05-19 10:07:30,744: yaml file:params.yaml load successfully: INFO
2024-05-19 10:07:30,748: Directory artifacts created successfully: INFO
2024-05-19 10:07:32,270: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.: WARNING


Found 8266 images belonging to 2 classes.


/home/fibonacci/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-05-19 10:07:34.903647: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-05-19 10:07:37.156847: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-05-19 10:07:39.041744: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51380224 exceeds 10% of free system memory.
2024-05-19 10:07:39.102762: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2024-05-19 10:07:40.15

 13/517 ━━━━━━━━━━━━━━━━━━━━ 2:37:58 19s/step - accuracy: 0.8860 - loss: 0.2891